En primer lugar preparamos el entorno importando las librerías que se van a utilizar

In [1]:
# Manejo de datos
import pandas as pd
import numpy as np

# Texto y similitud
from sklearn.metrics.pairwise import cosine_similarity

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Utilidades
import os
import pickle
from collections import Counter, defaultdict

# OpenAI embeddings
from openai import OpenAI
from dotenv import load_dotenv


En primer lugar realizamos la limpieza del csv resultante del formulario que se encuentra en la carpeta llamada "data"

In [13]:
# corrected relative path (ensure the 'data' folder is one level up from the notebook)
registros = pd.read_csv("../data/Graph-Match _ CIS 2026-I(1-5).csv", encoding="latin-1")

In [15]:
#eliminamos las columnas que están vacías
registros = registros.dropna(axis=1, how="all")
#eliminamos las filas que no aceptaron participar en la actividad
registros = registros[registros["¿Aceptas participar en esta actividad?"] != "No"]
registros.head(5)

,ID,Start time,Completion time,Email,Name,¿Aceptas participar en esta actividad?,¿ Cómo te llamas ?,¿ Cómo te pueden contactar ? Puedes escribir tu usuario de Instagram o tu número de Whatsapp.\n Esta información solo será visible para las personas con las que tengas match.,¿ Cuantos años tienes ?,Da una descripción de ti !! ¿ Cómo eres física y emocionalmente ? ?????????????? (Esto lo van a leer tus match),...,¿ Estas buscando algo serio ? ??????,¿ Qué tan extrovertido/a eres ?,¿ Qué tan emocional eres ?,¿ Que tan romántico/a eres ?,¿ Eres la cucharita grande o la cucharita pequeña ?,¿ Cómo expresas tu love lenguaje (lenguaje de amor) ? ?????,¿ Como te gusta recibir amor ? ?????,¿ Con qué género te identificas ?,¿Con qué tipo de personas te gustaría conectar?,Describe tu tipo ideal !! (Física y emocionalmente)
0,1,2/1/26 17:35:45,2/1/26 17:38:14,anonymous,NaN,NaN,TestUsuario,Test,NaN,Test,...,Si,5.0,5.0,NaN,NaN,NaN,NaN,Mujer,Hombres,Test
1,2,2/3/26 13:32:13,2/3/26 13:35:53,ambanos@uninorte.edu.co,ANGELICA MARIA BAÑOS PALLARES,NaN,Angélica,AngieIsTalking,18-20,Meow meow meow meow meow meow meow,...,Si,3.0,4.0,4.0,Pequeña,Tiempo de calidad;Actos de servicio;Palabras;,Tiempo de calidad;Actos de servicio;Contacto f...,Mujer,Hombres,Un femboy bien gótico ????????
2,3,2/5/26 9:07:28,2/5/26 9:16:39,sdariana@uninorte.edu.co,DARIANA SANGUINO CUELLO,NaN,Dariana Sanguino,3052333964,18-20,"Soy blanca, delgada, mido 1,63, me considero i...",...,Si,4.0,3.0,4.0,Pequeña,Tiempo de calidad;Contacto físico;Actos de ser...,Tiempo de calidad;Regalos;Contacto físico;Acto...,Mujer,Hombres,"Que me escuche, que este pendiente de mi, que ..."
3,4,2/5/26 9:08:58,2/5/26 9:30:53,amfrias@uninorte.edu.co,Adolfo Moises Frias Pardo,NaN,Adolfo Frías,@ado_drkc,18-20,"Soy delgado, cabello oscuro ni muy corto ni mu...",...,Si,3.0,3.0,4.0,Grande,Palabras;Tiempo de calidad;Contacto físico;Reg...,Palabras;Contacto físico;Tiempo de calidad;,Hombre,Mujeres,Alguien con quién pueda hablar de lo que sea y...


A continuación convertiremos cada fila/registro en un nodo

In [ ]:
usuarios = {}

for index, row in registros.iterrows():
   user = {
       "metadata":{
           "nombre": row["nombre"],
           "contacto": row["contacto"],
           "correo": row["Email"],
           "división": row["div"],
           "rango_edad": row["edad"],
           "género": row["genero"],
           "gen_interes": row["genero_interes"],
           "algo_serio": row["algo_serio"],
           "tipo_relacion": row["tipo"]
       },
       "var_estructurales": {
           "géneros_musica": row["musica"].split(";"),
           "planes_finde": row["finde"].split(";"),
           "tiempo_libre": row["freetime"].split(";"),
           "color": row["color"],
           
       },
       "escalas": {
           "nivel_extrovertido": row["extrover"],
           "nivel_emocional": row["emo"],
           "nivel_romantico": row["romantico"]
       },
       "love_and_roles": {
           "love_language_dar": row["love_language_dar"],
           "love_language_recibir": row["love_language_recibir"],
           "cucharita": row["cuchara"]
       },
       "arquetipo_visual": {
           "arquetipo_indentidad": row["me_identifico"],
           "arquetipo_ideal": row["me_gusta"],
           "pareja_favorita": row["pareja_fav"]
           
       },
       "descripcion": {
           "descripción_propia": row["mi_descripcion"],
           "descripción_tipo": row["mi_descripcion"]
       },
       "embeddings": {
           "propia": None,
           "tipo": None
       },
       "feedback": {
           "asistirá": row["asistencia"],
           "motivación": row["motivacion"]
       }
   }
   
# usamos el índice como ID único
usuarios[f"id_{index}"] = user

Una vez con cada persona convertida en un nodo, podemos pasar sus descripciones a vectores numéricos con el API de openai.

In [ ]:
load_dotenv()  # Carga las variables de entorno desde el archivo .env
openai = OpenAI(api_key=os.getenv("OPEN_AI_API_KEY"))   


In [17]:
def generar_embedding(texto, model="text-embedding-3-small"):
    """
    Recibe un texto y devuelve su embedding como lista de floats.
    """
    if texto is None or texto.strip() == "":
        return None

    response = openai.embeddings.create(
        model=model,
        input=texto
    )

    return response.data[0].embedding


In [ ]:
for user_id, user in usuarios.items():

    texto_propio = user["descripcion"]["descripción_propia"]
    texto_tipo = user["descripcion"]["descripción_tipo"]

    user["embeddings"]["propia"] = generar_embedding(texto_propio)
    user["embeddings"]["tipo"] = generar_embedding(texto_tipo)  

A continuación haremos un ciclo for, para realizar las comparaciones pertinentes, es importante tener en cuenta no realizar una comparación en bruto dado que es exactamente igual para este caso comparar a la persona A con la persona B que la persona B con la persona A; por lo tanto usaremos itertools, un módulo de python especializado en funciones que devuelven objetos iterables diseñado para una ejecución rápida haciendo uso eficiente de la memoria.

In [1]:
from itertools import combinations


In [5]:
personas = {
    "ID1": {"nombre": "Ana", "gustos": ["Cine", "Bici","coco","ron","vino","playa","azul"]},
    "ID2": {"nombre": "Beto", "gustos": ["Bici", "Libros","vino","agua","rio","azul"]},
    "ID3": {"nombre": "Carla", "gustos": ["Cine", "Libros"]}
}

In [ ]:
#ejemplo intersección:
# Intersección gustos (música) y hobbies (planes/tiempo libre)
gustos_comun = len(set(personas["ID1"]["gustos"]) & set(personas["ID2"]["gustos"]))
print(f"Gustos en común entre {personas['ID1']['nombre']} y {personas['ID2']['nombre']}: {gustos_comun}")

Gustos en común entre Ana y Beto: 3


In [ ]:
def calcular_match(u1, u2):
    # --- BLOQUE ESTRUCTURADO (45%) ---
    # Puntos max del bloque: 13+7+10+15 = 45 pts
    
    # Intersección gustos (música) y hobbies (planes/tiempo libre)
    gustos_comun = len(set(u1["var_estructurales"]["géneros_musica"]) & set(u2["var_estructurales"]["géneros_musica"]))
    hobbies_comun = len(set(u1["var_estructurales"]["planes_finde"]) & set(u2["var_estructurales"]["planes_finde"]))
    
    pts_gustos = min(gustos_comun, 13) # Tope de 13
    pts_hobbies = min(hobbies_comun, 7)  # Tope de 7
    
    # Love Languages (10 pts)
    # Cruce: lo que A da vs lo que B recibe Y viceversa
    pts_love = 0
    if u1["love_and_roles"]["love_language_dar"] == u2["love_and_roles"]["love_language_recibir"]:
        pts_love += 5
    if u2["love_and_roles"]["love_language_dar"] == u1["love_and_roles"]["love_language_recibir"]:
        pts_love += 5
        
    # Escalas (15 pts) - Diferencia normalizada
    # Promediamos la cercanía en las 3 escalas
    diffs = [
        abs(u1["escalas"]["nivel_extrovertido"] - u2["escalas"]["nivel_extrovertido"]),
        abs(u1["escalas"]["nivel_emocional"] - u2["escalas"]["nivel_emocional"]),
        abs(u1["escalas"]["nivel_romantico"] - u2["escalas"]["nivel_romantico"])
    ]
    # Si la escala es de 1 a 5, la dif max es 4. 
    # (1 - promedio_dif/4) nos da un porcentaje de cercanía.
    pts_escalas = (1 - (sum(diffs)/len(diffs)) / 4) * 15
    
    bloque_estructurado = pts_gustos + pts_hobbies + pts_love + pts_escalas

    # --- BLOQUE FOTOS / ARQUETIPOS (25%) ---
    # Puntos max: 10 + 10 + 5 = 25 pts
    pts_identidad = 0
    if u1["arquetipo_visual"]["arquetipo_indentidad"] == u2["arquetipo_visual"]["arquetipo_ideal"]:
        pts_identidad += 10
    if u2["arquetipo_visual"]["arquetipo_indentidad"] == u1["arquetipo_visual"]["arquetipo_ideal"]:
        pts_identidad += 10
    
    pts_pareja = 5 if u1["arquetipo_visual"]["pareja_favorita"] == u2["arquetipo_visual"]["pareja_favorita"] else 0
    
    bloque_fotos = pts_identidad + pts_pareja

    # --- BLOQUE TEXTO (30%) ---
    # Aquí asumo que ya tienes los embeddings o una función de similitud coseno
    # Si no los tienes, puedes usar un marcador de posición (placeholder)
    similitud_A_B = comparar_embeddings(u1["embeddings"]["propia"], u2["embeddings"]["tipo"])
    similitud_B_A = comparar_embeddings(u2["embeddings"]["propia"], u1["embeddings"]["tipo"])
    
    promedio_similitud = (similitud_A_B + similitud_B_A) / 2
    bloque_texto = promedio_similitud * 30 # Convertimos el 0.0-1.0 a escala de 30 pts

    # --- SCORE FINAL ---
    return bloque_estructurado + bloque_fotos + bloque_texto

In [ ]:
users_ids = list(usuarios.keys())

for id_a, id_b in combinations(users_ids, 2):
    print(id_a, "vs", id_b)
    u1 = usuarios[id_a]
    u2 = usuarios[id_b]
    # 1. FilTROS DE EXCLUSIÓN
    # a) género de interés vs género propio: Si a u1 le interesan hombres y u2 es mujer, saltamos, del mismo modo, si a u2 le interesan mujeres 
    # y u1 es hombre, también saltamos.
    interes_1_por_2 = (u1["metadata"]["gen_interes"] == u2["metadata"]["género"]) or (u1["metadata"]["gen_interes"] == "Me da igual")
    interes_2_por_1 = (u2["metadata"]["gen_interes"] == u1["metadata"]["género"]) or (u2["metadata"]["gen_interes"] == "Me da igual")   
    # Si uno de los dos no está interesado en el género del otro, abortamos el match
    if not (interes_1_por_2 and interes_2_por_1):
        continue
    elif u1["metadata"]["algo_serio"] != u2["metadata"]["algo_serio"]:
        # Borramos el match??? 'continue' o seguimos con menos puntos?????*************
        continue
    elif u1["metadata"]["rango_edad"] != u2["metadata"]["rango_edad"]:
        continue
    else:
    # --- CÁLCULO DE SCORE ---
        score = 0

ID1 vs ID2
ID1 vs ID3
ID2 vs ID3
